# Demo: Interactive Widgets
A notebook to explore the ipywidgets interactive widgets with clojupyter kernel.
The notebook explores widget creation. For examples of widget composition and interaction see interactive-widgets.ipynb

Requires clojupyter-0.3.3.-snapshot.

In [ ]:
(require '[clojupyter.widgets.alpha :as alpha])
(require '[clojupyter.widgets.ipywidgets :as ipy])
(require '[clojure.string :as s]);

## Simple Widgets
### Text Widgets
Let's build a text widget that returns the reverse upper case of the input string.
The function bellow achieves that:

In [ ]:
(ipy/label {:value "My text widget"})

In [ ]:
(ipy/text {:placeholder "Hello World"})

In [ ]:
(ipy/password {:value "top secret pwd" :description "Password"})

In [ ]:
(ipy/textarea {:placeholder "Some lengthy description" :description "Your Text Here:" :rows 4})

In [ ]:
(ipy/combobox {:options ["blue" "black" "green" "yellow"] :description "Pick a color"})

Passing the fn directly to the widget does not work, because the fn passed to interactive needs to take a hash-map argument.

In [ ]:
(ipy/html {:value "<p>Hello <b>World</b></p>" :placeholder "Some HTML" :description "Some HTML"})

Example taken from: https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html#HTML-Math

In [ ]:
(ipy/html-math {:value "Some math and <i>HTML</i>: \\(x^2\\) and $$\\frac{x+1}{x-1}$$"})

Note: *We need to escape the backslash in order for the above example to work.*

____
### Boolean Widgets


In [ ]:
(ipy/checkbox {:description "Click Me" :value true})

In [ ]:
(ipy/toggle-button {:value false :description "Click to Activate" :icon "hand-pointer"})

In [ ]:
(ipy/valid {:value true :description "Valid!"})

___
### Numeric Widgets

In [ ]:
(ipy/int-slider {:description "Vertical Slider" :value 20 :min -10 :max 20
                         :orientation "vertical"})

In [ ]:
(ipy/float-slider {:description "Horizontal Slider" :value (rand) :min 0.0 :max 1.0
                           :step 0.01 :readout_format ".2f"})

In [ ]:
(ipy/float-log-slider {:value 2e5 :base 10.0 :min 4.0 :max 8.0 :step 0.2})

In [ ]:
(ipy/int-range-slider {:value [200 1800] :max 3000 :step 5})

Note: *Passing the type fn to most (all?) widgets kills the kernel.*

In [ ]:
(ipy/float-range-slider)

In [ ]:
(ipy/int-progress {:min 10 :max 100 :value 95})

In [ ]:
(ipy/float-progress {:min 0.0 :max 10.0 :step 0.1 :value 4.8})

Note: *How does one animate the above progress bars?*

In [ ]:
(ipy/bounded-int-text {:min 0 :max 100 :step 5})

Note: *The above widget acts like a bounded-float-text if we pass a float step.*
No longer true, now that the widgets get validated.

In [ ]:
(ipy/bounded-float-text {:min 0.0 :max 10.0 :step 0.2})

In [ ]:
(ipy/int-text {:value 233})

In [ ]:
(ipy/float-text {:value 9.33 :min 0.01 :step 0.02})

____
### Selection Widgets

In [ ]:
(def A (ipy/dropdown {:options ["one" "two" "three"] :value "three"}))
A

In [ ]:
(ipy/dropdown {:options {:one 1 :two 2 :three 3} :index 2})

In [ ]:
(ipy/dropdown {:options [["one" 1] ["two" 2] ["three" 3]] :index 2 :description "Pick a number"})

In [ ]:
(ipy/box {:children [(ipy/radio-buttons {:options ["dark" "light"] :description "A long description here" :value "dark"})]})

In [ ]:
(ipy/select {:options ["english" "french" "german"] :value "german" :description "Select Language"})

In [ ]:
(ipy/toggle-buttons {:options ["english" "french" "german"] :value "french" :description "A very looooooooooooooooooooooooooong description" :button_style "primary" :disabled true
                     :style (ipy/toggle-buttons-style {:button_width "100px"})})

In [ ]:
(ipy/select-multiple {:options ["Guitar" "Mandolin" "Violin" "Bass"] :value ["Bass"] :description "String Instruments"})

In [ ]:
(ipy/selection-slider {:options ["slower" "slow" "normal" "fast" "fastest"] :value "normal" :description "Speed"})

In [ ]:
(ipy/selection-range-slider {:options (map list ["slower" "slow" "normal" "fast" "fastest"] (range)) :value [2 2] :description "Speed"})

____
### Container Widgets

In [ ]:
(let [w (ipy/int-slider)
      p (ipy/int-slider {:max (* 2 (:max @w))})
      _ (ipy/directional-link {:source [w :value] :target [p :value]})]
  (ipy/box {:children [w p]}))

In [ ]:
(let [w (ipy/int-slider {:orientation "vertical" :value (rand-int 101)})
      p (ipy/int-progress {:orientation "vertical" :value (:value @w)})]
  (.watch w :key0 (fn [_ _ _ new-state] (swap! p assoc :value (:value new-state))))
  (ipy/h-box {:children [w p]}))

H-box looks and feels identical with a regular box widget.

In [ ]:
(let [w (ipy/int-slider)
      p (ipy/int-progress)]
  (.watch w :key0 (fn [_ _ _ new-state] (swap! p assoc :value (:value new-state))))
  (ipy/v-box {:children [w p]}))

In [ ]:
(let [w (ipy/label {:value "Hello"})
      p (ipy/label {:value "World"})]
  (ipy/accordion {:children [w p]}))

In [ ]:
(let [w (ipy/label {:value "Hello"})
      p (ipy/label {:value "World"})]
  (ipy/tab {:children [w p] :selected_index 0}))

In [ ]:
(let [l0 (ipy/label {:value "Hello"})
      l1 (ipy/label {:value "World"})
      s (ipy/stacked {:children [l0 l1] :selected_index 0})
      w0 (ipy/radio-buttons {:options ["Stack 0" "Stack 1"] :description "Select stack" :index 0})]
  (.watch w0 :key0 (fn [_ _ _ new-state] (swap! s assoc :selected_index (:index new-state))))
  (ipy/box {:children [w0 s]}))

____
### Other Widgets

In [ ]:
(require '[clojure.java.io :as io])
(defn slurp-bytes
  [^String filename & opt]
  (let [file (io/file filename)
        buf (byte-array (.length file))]
    (with-open [fis (io/make-input-stream file (when opt (apply hash-map opt)))]
      (.read fis buf))
    buf))

In [ ]:
(let [img (slurp-bytes "../resources/clojupyter/assets/logo-64x64.png")]
  (ipy/image {:value img :format "png" :height "200" :width "120"}))

In [ ]:
(ipy/button {:description "Click me" :on-click (constantly nil)}) 

In [ ]:
(let [min 0
      max 100
      value 30
      step 1
      w (ipy/play {:value value :min min :max max :step step :interval 500})
      prg (ipy/int-progress {:min min :max max :value value})]
  (.watch w :key0 (fn [_ _ _ new-state] (swap! prg assoc :value (:value new-state))))
  (ipy/h-box {:children [w prg]}))

In [ ]:
(ipy/date-picker {:description "Pick a date" :disabled false})

In [ ]:
(ipy/color-picker {:description "Pick a color" :value "blue" :concise false})

In [ ]:
(def fu (ipy/file-upload))
fu

In [ ]:
(let [X (ipy/controller-axis)
      B (ipy/controller-button)]
  (ipy/controller {:axes [X] :buttons [B]}))